In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from estudo import Estudo

df, erros = Estudo(
    ano = 2024,
    id_modulo = None,
    analito = None
).run()

In [10]:
df[['id_modulo','modulo','analito']].drop_duplicates()

,id_modulo,modulo,analito
0,58,Bioquímica I,Albumina
28,58,Bioquímica I,Amilase Total
45,58,Bioquímica I,Bilirrubina Direta
1646,58,Bioquímica I,Bilirrubina Indireta
1658,58,Bioquímica I,Bilirrubina Total
...,...,...,...
15824,684,Etanol Combustíveis: pH,pH (25°C)
15828,691,Suco de Frutas: pH,pH
15831,1068,POCT Marcadores Tumorais I (N4),PSA Total Quanti
15835,1091,POCT Bioquímica I (N4),Colesterol Total


In [2]:
from utils import Data

data = Data()
# parceiros = data.get_parceiros()
# modulos = data.get_modulos()
df = data.get_data(ano = 2024, id_modulo = [362], analito = ['Fosfatase Ácida Total'])

from utils import PreProcessamento

df_tratado = PreProcessamento(df).get_data()
df_tratado

item,part,id_modulo,analito,sistema,A24M10I1,A24M10I2,A24M10I3,A24M1I1,A24M1I2,A24M1I3,A24M4I1,A24M4I2,A24M4I3,A24M7I1,A24M7I2,A24M7I3
0,25,362,Fosfatase Ácida Total,1,17.6,4.2,5.3,8.9,9.7,3.8,2.4,5.5,3.3,9.4,2.8,3.7
1,94,362,Fosfatase Ácida Total,1,16.9,3.5,4.9,8.2,9.0,2.8,3.0,5.7,3.9,9.0,3.1,3.6
2,124,362,Fosfatase Ácida Total,1,16.8,3.4,4.2,6.6,7.6,2.7,2.3,4.6,2.7,8.6,2.9,3.4
3,124,362,Fosfatase Ácida Total,2,17.9,4.0,5.2,7.7,9.4,3.2,2.7,5.8,3.7,8.6,3.1,3.5
4,3466,362,Fosfatase Ácida Total,1,NaN,NaN,NaN,8.8,8.9,2.7,2.4,5.6,3.2,8.3,2.5,3.2
5,5596,362,Fosfatase Ácida Total,1,16.6,3.6,4.3,7.5,8.8,2.8,2.1,4.7,3.3,8.5,2.8,3.2
6,6086,362,Fosfatase Ácida Total,1,18.3,3.7,5.0,6.7,7.8,2.1,2.5,5.3,3.0,8.4,3.0,3.2
7,7463,362,Fosfatase Ácida Total,1,NaN,NaN,NaN,7.6,8.5,3.1,2.3,5.3,3.3,8.6,2.9,3.4
8,9379,362,Fosfatase Ácida Total,1,15.9,3.4,4.3,7.5,8.4,3.0,2.5,5.1,3.3,8.0,3.1,3.2
9,9466,362,Fosfatase Ácida Total,1,17.9,4.0,5.2,7.7,9.4,3.2,2.7,5.8,3.7,8.6,3.1,3.5


In [3]:
teste = df_tratado.set_index(['part','id_modulo','analito','sistema'])

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler    
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Preenchendo vazios pela média da linha
imputer = SimpleImputer(strategy="mean")
dados: np.ndarray = imputer.fit_transform(teste.T).T

# Aplicando a normalização
norm = StandardScaler()

dados = norm.fit_transform(dados)
pd.DataFrame(dados)

neighbors = NearestNeighbors(n_neighbors=2)
neighbors_fit = neighbors.fit(dados)
distances, _ = neighbors_fit.kneighbors(dados)
distances = np.mean(distances[:,1:], axis=1)
distances = np.sort(distances)
distances = distances[np.where(distances <= 2)]
slopes = np.diff(distances)
value = slopes[np.where(slopes>=1/100)][[0]]
value
# slopes[np.where(slopes>=1/100)]

index = np.argmax(slopes==value)

eps = distances[index+1]
eps

IndexError: index 0 is out of bounds for axis 0 with size 0